In [8]:
from Crypto.Cipher import AES
from random import choice
from prob import *
import re
import base64

PREFIX = list('佛曰：')
# with open("flag1","r",encoding="utf-8") as f:
#     x=f.read()
# with open("flag1.enc","w",encoding="utf-8") as f:
#     f.write(Encrypt(x).encode("utf-8").decode("shift_jis",errors="ignore"))

In [9]:
# read flag1
with open('flag1.enc',encoding='utf-8') as f:
    raw = f.read()

jis = raw.encode('shift_jis')
# patch PREFIX, since we know this cipher well
prefix_bytes = ''.join(PREFIX).encode('utf-8')
prefix_len_after_jis = 0
for b in prefix_bytes:
    if jis[prefix_len_after_jis] != b:
        continue
    prefix_len_after_jis += 1
jis = jis[prefix_len_after_jis:]

jis_sep = [c.encode('shift_jis') for c in raw]
jis_sep_len = [len(c.encode('shift_jis')) for c in raw]


In [10]:
# gain knowledge of TUDOU, BYTEMARK, PREFIX in shift_jis
# valid table: nx3 tuple c:v[i][j]=ith byte validate in JIS:(1Byte,1st2Byte,2nd2Byte)[j]
def is_valid_JIS(b):
    try:
        b.decode('shift_jis')
        return True
    except UnicodeDecodeError:
        return False

one_jis_pool = [c for c in range(256) if (c <= 0x7f) or (c >= 0xa1 and c <= 0xdf) ]
first_jis_pool = [c for c in range(256) if (c >= 0x81 and c <= 0x9f) or (c >= 0xe0 and c <= 0xef)]
second_jis_pool = [c for c in range(256) if (c >= 0x40 and c <= 0x7e) or (c >= 0x80 and c <= 0xfc)]
no_jis_pool = list(set(range(256)).difference(one_jis_pool).difference(first_jis_pool))
# some second byte validity change with first byte, the ones with E included is important
invalid_2B_word = [(i,j) for i in first_jis_pool for j in second_jis_pool if is_valid_JIS((i*256+j).to_bytes(2,'big'))]
possible_invalid_first_set = {hex(s[0])  for i in range(0x40,0x100) for s in invalid_2B_word if s[1]==i}
e1_invalid_list = [(i*256+j).to_bytes(2,'big') for i in range(0xe0,0xf0) for j in range(256) if not is_valid_JIS((i*256+j).to_bytes(2,'big'))]
e2_invalid_list = [(i*256+j).to_bytes(2,'big') for i in range(256) for j in range(0xe0,0xf0) if not is_valid_JIS((i*256+j).to_bytes(2,'big'))]
e12_invalid_list = sorted(set(e1_invalid_list).intersection(e2_invalid_list))
# 2+1 form not in db
firsttwo_invalid_list = [c.encode('utf-8') for c in (TUDOU + BYTEMARK) if not is_valid_JIS(c.encode('utf-8')[:2])]
assert not firsttwo_invalid_list
lasttwo_invalid_list = [c.encode('utf-8') for c in (TUDOU + BYTEMARK) if not (is_valid_JIS(c.encode('utf-8')[1:]) or is_valid_JIS(c.encode('utf-8')[1].to_bytes(1,'big')))]
print('last two invalid:', lasttwo_invalid_list)

byte_valid_JIS = {c:(
    1 if c in one_jis_pool else\
         (2 if c in first_jis_pool else\
            0),
    1 if (c >= 0x40 and c<= 0x7e) or (c >= 0x80 and c <= 0xfc) else 0
) for c in range(256)}

TUDOU_valid = {i:tuple(byte_valid_JIS[b] for b in TUDOU[i].encode('utf-8')) for i in range(128)}
BYTEMARK_valid = {i:tuple(byte_valid_JIS[b] for b in BYTEMARK[i].encode('utf-8')) for i in range(len(BYTEMARK))}
PREFIX_valid = {i:tuple(byte_valid_JIS[b] for b in PREFIX[i].encode('utf-8')) for i in range(len(PREFIX))}
# all bytes are valid 2nd2Byte
assert all(s[i][1] for s in TUDOU_valid.values() for i in range(3))
# all invalid byte appears once in one character
# only invalid mark is BYTEMARK[6]
# => no consecutive lack bytes
# => byte before lack byte is in the same character
lack_byte_list = [i for i in range(128) if 0 in [t[0] for t in TUDOU_valid[i]]]

# set(range(0x40,256)).difference(set([c[1] for c in invalid_2B_word]))
len(e12_invalid_list)


last two invalid: [b'\xe8\x80\xb6', b'\xe9\x83\xbd', b'\xe7\x88\x8d', b'\xe6\x80\x9b', b'\xe8\x80\x85', b'\xe8\x98\x87', b'\xe9\x86\xaf', b'\xe8\x88\x8d', b'\xe9\x81\xae', b'\xe9\x80\x9d', b'\xe6\xa0\x97', b'\xe5\x88\x87', b'\xe8\x83\xbd', b'\xe6\x98\x8e', b'\xe8\x80\xa8', b'\xe5\x80\x92', b'\xe6\x80\x96', b'\xe3\x80\x82', b'\xe8\x80\x81', b'\xe8\x87\xb3', b'\xe5\x86\xa5', b'\xe6\x80\xaf']


96

In [11]:
# edge of TUDOU, BYTEMARK, PREFIX in shift_jis
# valid table: nx3 tuple c:v[i][j]=ith byte validate in JIS:(1Byte,1st2Byte,2nd2Byte)[j]
one_jis_pool = [c for c in range(256) if (c <= 0x7f) or (c >= 0xa1 and c <= 0xdf) ]
first_jis_pool = [c for c in range(256) if (c >= 0x81 and c <= 0x9f) or (c >= 0xe0 and c <= 0xef)]
no_jis_pool = list(set(range(256)).difference(one_jis_pool).difference(first_jis_pool))

byte_valid_JIS = {c:(
    1 if c in one_jis_pool else\
         (2 if c in first_jis_pool else\
            0),
    1 if (c >= 0x40 and c<= 0x7e) or (c >= 0x80 and c <= 0xfc) else 0
) for c in range(256)}

TUDOU_valid = {i:tuple(byte_valid_JIS[b] for b in TUDOU[i].encode('utf-8')) for i in range(128)}
BYTEMARK_valid = {i:tuple(byte_valid_JIS[b] for b in BYTEMARK[i].encode('utf-8')) for i in range(len(BYTEMARK))}
PREFIX_valid = {i:tuple(byte_valid_JIS[b] for b in PREFIX[i].encode('utf-8')) for i in range(len(PREFIX))}
# all bytes are valid 2nd2Byte
assert all(s[i][1] for s in TUDOU_valid.values() for i in range(3))
# all invalid byte appears once in one character
# only invalid mark is BYTEMARK[6]
# => no consecutive lack bytes
# => byte before lack byte is in the same character
lack_byte_list = [i for i in range(128) if 0 in [t[0] for t in TUDOU_valid[i]]]
[c.encode('utf-8') for c in BYTEMARK]

[b'\xe5\x86\xa5',
 b'\xe5\xa5\xa2',
 b'\xe6\xa2\xb5',
 b'\xe5\x91\x90',
 b'\xe4\xbf\xb1',
 b'\xe5\x93\x86',
 b'\xe6\x80\xaf',
 b'\xe8\xab\xb3',
 b'\xe7\xbd\xb0',
 b'\xe4\xbe\x84',
 b'\xe7\xbc\xbd',
 b'\xe7\x9a\xa4']

In [12]:
# from UTF-8 to Fo-str
def get_possible_byte(recent_char, next_char=None):
    res = []
    for i in lack_byte_list:
        c = TUDOU[i].encode('utf-8')
        if TUDOU_valid[i][1][0] == 0:
            is_second_byte = True
        elif TUDOU_valid[i][2][0] == 0:
            is_second_byte = False
        else:
            raise ValueError('wrong with lack byte list')
        
        if is_second_byte:
            possible = (c[:1] == (recent_char[-1:]))
            if next_char is not None:
                possible = possible and (c[-1] == next_char)
        else:
            possible = (c[:2] == (recent_char[-2:]))
            
        if possible:
            lack_byte = c[1] if is_second_byte else c[2]
            res.append(lack_byte)
    # TODO consider BYTEMARK

    assert len(res) == 1
    # if len(res)>1:
    #     print('warning: multiple possible char:', recent_char[-2:], res, next_char)

    return res


In [13]:
# split jis by \xe?
assert all(hex(c.encode('utf-8')[0])[2]=='e' for c in TUDOU)
assert all(hex(c.encode('utf-8')[0])[2]=='e' for c in BYTEMARK)
assert all(hex(c.encode('utf-8')[1])[2]!='e' for c in TUDOU)
assert all(hex(c.encode('utf-8')[2])[2]!='e' for c in TUDOU)
assert all(hex(c.encode('utf-8')[1])[2]!='e' for c in BYTEMARK)
assert all(hex(c.encode('utf-8')[2])[2]!='e' for c in BYTEMARK)

jis_split = []
jis_tosplit = jis
while jis_tosplit:
    i = 0
    while True:
        i += 1
        try:
            b = jis_tosplit[i]
        except IndexError:
            jis_split.append(jis_tosplit)
            jis_tosplit = []
            break

        if hex(b)[2]=='e':
            jis_split.append(jis_tosplit[:i])
            jis_tosplit = jis_tosplit[i:]
            break

for word in jis_split:
    # assert len(word)>=2 and len(word) <= 3
    if len(word)==3:
        assert word.decode('utf-8') in TUDOU + BYTEMARK
        print(word)
    else:
        print(word)
        
assert b''.join(jis_split) == jis

b'\xe9\xba\xbc'
b'\xe6\x98\xaf'
b'\xe6\xa2\xb5'
b'\xe6\xae\xbf'
b'\xe7\x9a\xa4'
b'\xe7\x9b\xa1'
b'\xe7\x9a\xa4'
b'\xe6\xbb'
b'\xe5\xa8\x91'
b'\xe8\x8b\xa6'
b'\xe5\x86\xa5'
b'\xe9\x82\xa3'
b'\xe7\x9b\xa7'
b'\xe7\xbe\xaf'
b'\xe4\xbc\x8a'
b'\xe7\xbd\xb0'
b'\xe6\x99\xba'
b'\xe7\x9b\xa1'
b'\xe5\xa4\x9c'
b'\xe6\xa2\xb5'
b'\xe5\x8b\x9d'
b'\xe4\xbf\xb1'
b'\xe5\xaf\xa6'
b'\xe5\x88\xa9'
b'\xe7\x9b\xa7'
b'\xe7\x9a\xa4'
b'\xe5\x91\xbc'
b'\xe5\x91\x90'
b'\xe3\x82'
b'\xe5\xb8\x9d'
b'\xe5\xa4\xa2'
b'\xe6\xa2\xb5'
b'\xe6\xbb'
b'\xe7\xbe'
b'\xe5\xaf\xa6'
b'\xe5\x91\x90'
b'\xe7\xad\x89'
b'\xe4\xbe\x9d'
b'\xe7\x9a\xa4'
b'\xe8\x98'
b'\xe4\xbe'
b'\xe5\x80\x92'
b'\xe4\xbc\x8a'
b'\xe4\xbb\xa5'
b'\xe4\xba\xa6'
b'\xe5\x93'
b'\xe6\x95\xb8'
b'\xe6\x80\xaf'
b'\xe5\xa7\xaa'
b'\xe5\xb0\xbc'
b'\xe6\x80\xaf'
b'\xe7\x84\xa1'
b'\xe5\x91\x90'
b'\xe6\x8e'
b'\xe5\x92'
b'\xe7\xa5\x9e'
b'\xe6\xaf'
b'\xe5\xa4\xb7'
b'\xe9\x81'
b'\xe4\xbe'
b'\xe6\x91\xa9'
b'\xe6\x99\xba'
b'\xe5\x91\x90'
b'\xe4\xbc\xbd'
b'\xe6\x81\x90'
b'\xe6\x

In [27]:
# 
jis_copy = jis[2:]
raw_copy = raw[6:]
# print(jis_copy)
# print(raw_copy.encode('shift_jis'))
i=0
jis_fixed = []
current_choice = []
revert_flag = False
while i < len(jis_split):
    line = jis_split[i]
    try:
        if len(line) == 3:
            assert line.decode('utf-8') in (TUDOU + BYTEMARK)
            assert (b''.join(jis_fixed) + line)[2:].decode('shift_jis',errors='ignore')[:] in raw_copy
            jis_fixed.append(line)

        elif len(line) == 2:
            possible_patch = [c.encode('utf-8') for c in (TUDOU + BYTEMARK) if (c.encode('utf-8')[:2]==line) or (c.encode('utf-8')[:3:2]==line)]
            possible_patch = [c for c in possible_patch if (b''.join(jis_fixed) + c)[2:].decode('shift_jis',errors='ignore')[:] in raw_copy]
            # if i==59:
            #     possible_patch.append(b'\xe4\xbe')
            print(i,line,possible_patch)
            jis_fixed.append(possible_patch[-1])
            current_choice.append((i, possible_patch))
        elif len(line) == 1:
            # TODO: or maybe not
            ''
        i+=1
    except:
        last_choice = current_choice.pop()
        while len(last_choice[1]) <= 1:
            last_choice = current_choice.pop()
            # print(current_choice)
        last_choice[1].pop()
        i = last_choice[0]
        jis_fixed = jis_fixed[:i+1]
        jis_fixed[i] = last_choice[1][-1]
        current_choice.append(last_choice)
        # revert_flag = True

        i+=1

# print(jis_fixed, current_choice)
[len(c[1]) for c in current_choice]
# print('\n'.join([str(l) for l in current_choice]))
# print(current_choice[11])
i = 0
j = 0
# parameter_stack = [0] * current_choice
for i in range(2):
    for j in range(2):
        for k in range(5):
            # print(i,j,k)
            jis_fixed[current_choice[5][0]] = current_choice[5][1][i]
            jis_fixed[current_choice[11][0]] = current_choice[11][1][j]
            jis_fixed[current_choice[-1][0]] = current_choice[-1][1][k]
        

            try:
                cip = '佛曰：'+b''.join(jis_fixed).decode('utf-8')
                print(Decrypt(cip))
            except:
                ''



7 b'\xe6\xbb' [b'\xe6\xbb\x85']
28 b'\xe3\x82' [b'\xe3\x80\x82']
32 b'\xe6\xbb' [b'\xe6\xbb\x85']
33 b'\xe7\xbe' [b'\xe7\xbe\x85']
39 b'\xe8\x98' [b'\xe8\x98\x87']
40 b'\xe4\xbe' [b'\xe4\xbe\x9d', b'\xe4\xbe\x84']
45 b'\xe5\x93' [b'\xe5\x93\x86']
53 b'\xe6\x8e' [b'\xe6\x98\x8e']
54 b'\xe5\x92' [b'\xe5\x80\x92']
56 b'\xe6\xaf' [b'\xe6\x80\xaf']
58 b'\xe9\x81' [b'\xe9\x81\xa0', b'\xe9\x81\x93']
59 b'\xe4\xbe' []
59 b'\xe4\xbe' [b'\xe4\xbe\x9d', b'\xe4\xbe\x84']
65 b'\xe6\xaf' [b'\xe6\x80\xaf']
70 b'\xe4\xb8' [b'\xe4\xb8\x8d', b'\xe4\xb8\x80', b'\xe4\xb8\x8a', b'\xe4\xb8\x89', b'\xe4\xb8\x96']
flag{s1mp1e_Tud0uc0d3}
flag{s1mp1e_Tud0銲ꅙ揲锔睟챽ᄻ
flag{s1mp1e_Tud0룄爉鸆몠㊆硎ữ
flag{s1mp1e_Tud0⧶䳐鑅绎箄䰍໎


In [ ]:
# 
jis_copy = jis[2:]
raw_copy = raw[6:]
# print(jis_copy)
# print(raw_copy.encode('shift_jis'))
i=0
jis_fixed = []
current_choice = []
revert_flag = False
while i < len(jis_split):
    line = jis_split[i]
    try:
        if len(line) == 3:
            assert line.decode('utf-8') in (TUDOU + BYTEMARK)
            assert (b''.join(jis_fixed) + line)[2:].decode('shift_jis',errors='ignore')[:-1] in raw_copy
            jis_fixed.append(line)

        elif len(line) == 2:
            if not revert_flag:
                possible_patch = [c.encode('utf-8') for c in (TUDOU + BYTEMARK) if (c.encode('utf-8')[:2]==line) or (c.encode('utf-8')[:3:2]==line)]
                possible_patch = [c for c in possible_patch if (b''.join(jis_fixed) + c)[2:].decode('shift_jis',errors='ignore')[:-1] in raw_copy]
                # if i==59:
                #     possible_patch.append(b'\xe4\xbe')
                print(i,line,possible_patch)
                jis_fixed.append(possible_patch[-1])
                current_choice.append((i, possible_patch))
            else:
                possible_patch = current_choice[-1][1]
                jis_fixed.append(possible_patch[-1])
        i+=1
    except:
        last_choice = current_choice.pop()
        while len(last_choice[1]) <= 1:
            last_choice = current_choice.pop()
        last_choice[1].pop()
        i = last_choice[0]
        jis_fixed = jis_fixed[:i]
        current_choice.append(last_choice)
        revert_flag = True

        i+=1

print(jis_fixed, current_choice)


7 b'\xe6\xbb' [b'\xe6\xbb\x85']
28 b'\xe3\x82' [b'\xe3\x80\x82']
32 b'\xe6\xbb' [b'\xe6\xbb\x85']
33 b'\xe7\xbe' [b'\xe7\xbe\xaf', b'\xe7\xbe\x85']
39 b'\xe8\x98' [b'\xe8\x98\x87']
40 b'\xe4\xbe' [b'\xe4\xbe\x9d', b'\xe4\xbe\x84']
45 b'\xe5\x93' [b'\xe5\x93\x86']
53 b'\xe6\x8e' [b'\xe6\x98\x8e']
54 b'\xe5\x92' [b'\xe5\x92\x92', b'\xe5\x80\x92']
56 b'\xe6\xaf' [b'\xe6\x98\xaf', b'\xe6\x80\xaf']
58 b'\xe9\x81' [b'\xe9\x81\xae', b'\xe9\x81\xa0', b'\xe9\x81\x93']
59 b'\xe4\xbe' []


In [ ]:

def is_e(b):
    '''only first byte of UTF-8 is 0xe?
    '''
    return base64.b16encode(b).decode()[0] == 'E'

def get_possible_insert(known, pos, db=None):
    '''check whether there are missed bytes
    known: 2Bytes known
    pos: unknown pos
    db: where to search. TUDOU, BYTEMARK
    '''
    if db is None:
        db = TUDOU + BYTEMARK
    if pos == 1:
        check_list = [known[0] + i.to_bytes(1, 'big') + known[1] for i in no_jis_pool]
    elif pos == 2:
        check_list = [known[:2] + i.to_bytes(1, 'big') for i in no_jis_pool]
    else:
        raise ValueError(pos)
    
    return [b for b in check_list if b.decode('utf-8',errors='ignore') in db]

assert not any(is_e(i.to_bytes(1, 'big')) for i in no_jis_pool)



jis_sep_copy = jis_sep[7:]
# stack of result
# (i, fo_buf, )
fo_stack = []
fo_buf = b''
unknown_cip = []
i = 0
while i < len(jis_sep_copy):
    sep = jis_sep_copy[i]
    try:
        if len(fo_buf) == 0:
            # no possible first two/first one insert!

            # lack character cannot be e
            if is_e(sep[0].to_bytes(1,'big')):
                fo_buf += sep
                # stacktrace:
                trace = [((i+1), fo_buf, sep)]

        elif len(fo_buf) == 2:
            # insert, fo_buf become empty
            # # no insert might be consecutive! 2+2
            possible_third_byte = [c.encode('utf-8')[2] for c in (TUDOU + BYTEMARK) if c.encode('utf-8')[:2]==fo_buf]
            # insert 1 invalid:
            insert_list = get_possible_insert(fo_buf, 2)
            trace = [(i, b'', b) for b in insert_list]
            # insert 2 invalid (2+2):
            possible_2B = [b for b in e2_invalid_list if b[0] in possible_third_byte]
            trace += [(i, b[1].to_bytes(1,'big'), b) for b in possible_2B]

            # 1B, 2B
            this_sep, next_sep = sep[:1], sep[1:]
            # this_char = (fo_buf + this_sep).decode('utf-8', errors='ignore')
            if this_sep in possible_third_byte:
                # possible trace
                if len(sep) == 1:
                    trace += [(i+1, b'', sep)]
                else:
                    # 2+2
                    if is_e(next_sep):
                        trace += [(i+1, next_sep, sep)]

        elif len(fo_buf) == 1:
            
            accept_next_2B = [c.encode('utf-8')[1:] for c in (TUDOU+BYTEMARK) if c.encode('utf-8')[:1]==fo_buf]
            # insert: 
            # 2Byte: 1+2
            insert_2B = [b for b in accept_next_2B if b in lasttwo_invalid_list]
            trace = [(i, b'', b) for b in insert_2B]
            # # TODO 1+1+2 may be possible?
            # insert_112 = sorted(set(b1 for b1 in no_jis_pool for b2 in e2_invalid_list if (b1 + b2[0]) in accept_next_2B))
            # trace = [(i, b'', b) for b in insert_2B]
            # insert 1Byte: too complicated
            insert_list = [b for b in no_jis_pool if b in [c[0] for c in accept_next_2B]]
            trace += [(i, fo_buf + b.to_bytes(1,'big'), b) for b in insert_list]
            # sep == 2, complete cycle
            if len(sep) == 2:
                fo_buf += sep
                c = fo_buf.decode('utf-8')
                # TODO: handle exception
                if (c in TUDOU) or (c in BYTEMARK):
                    trace += [(i+1, b'', sep)]
            else:
                # 1+1,
                trace += [(i+1, fo_buf + sep, sep)]

        # if no possible trace
        assert trace

    except AssertionError:
        # backtrace, discard current trace
        trace = None
        while (not trace):
            trace = fo_stack.pop()
            trace.pop()

    finally:
        # update local variable, push stack
        i = trace[-1][0]
        fo_buf = trace[-1][1]
        fo_stack.append(trace)
        print(fo_stack)
        trace = None





In [23]:
print(is_e(b'\xef'))
jis[18:]

True


b'\xe7\x9a\xa4\xe6\xbb\xe5\xa8\x91\xe8\x8b\xa6\xe5\x86\xa5\xe9\x82\xa3\xe7\x9b\xa7\xe7\xbe\xaf\xe4\xbc\x8a\xe7\xbd\xb0\xe6\x99\xba\xe7\x9b\xa1\xe5\xa4\x9c\xe6\xa2\xb5\xe5\x8b\x9d\xe4\xbf\xb1\xe5\xaf\xa6\xe5\x88\xa9\xe7\x9b\xa7\xe7\x9a\xa4\xe5\x91\xbc\xe5\x91\x90\xe3\x82\xe5\xb8\x9d\xe5\xa4\xa2\xe6\xa2\xb5\xe6\xbb\xe7\xbe\xe5\xaf\xa6\xe5\x91\x90\xe7\xad\x89\xe4\xbe\x9d\xe7\x9a\xa4\xe8\x98\xe4\xbe\xe5\x80\x92\xe4\xbc\x8a\xe4\xbb\xa5\xe4\xba\xa6\xe5\x93\xe6\x95\xb8\xe6\x80\xaf\xe5\xa7\xaa\xe5\xb0\xbc\xe6\x80\xaf\xe7\x84\xa1\xe5\x91\x90\xe6\x8e\xe5\x92\xe7\xa5\x9e\xe6\xaf\xe5\xa4\xb7\xe9\x81\xe4\xbe\xe6\x91\xa9\xe6\x99\xba\xe5\x91\x90\xe4\xbc\xbd\xe6\x81\x90\xe6\xaf\xe4\xba\xa6\xe5\x86\xa5\xe5\xbd\x8c\xe7\xbc\xbd\xe4\xb8'

In [ ]:

# fix incomplete bytes
incomplete_index = [i for i in range(len(jis_split)) if len(jis_split[i]) < 3]
incompete_patch = []
for iL in incomplete_index:
    line = jis_split[iL]
    possible_fix = []
    # for c in TUDOU + BYTEMARK:
    for c in ([TUDOU[i] for i in lack_byte_list] + [BYTEMARK[6]]):
        c_byte = c.encode('utf-8')
        if (c_byte[0] == line[0]) and ((c_byte[1] == line[1]) or (c_byte[2] == line[1])):
            # validate that this would yield UnicodeDecodeError
            try:
                c_byte.decode('shift_jis')
            except UnicodeDecodeError:
                possible_fix.append(c_byte)
    if len(possible_fix)==0:
        print(line)
    incompete_patch.append(possible_fix)
    # assert len(possible_fix) > 0 
    
incompete_patch

In [ ]:
# fo_cip = ''
# jis_buf = b''
fo_buf = [b'']
inserted_character_stack = []
jis_fixed = jis

i = 0
while i < len(jis_fixed):
    b = jis_fixed[i]
    # jis_buf += b

    if len(fo_buf[-1]) == 0:
        # first hex must be e, else there must be lack of bytes
        # lack bytes must be in no_jis_pool
        if(str(b)[2] != 'e'):
            if i < len(jis_fixed) - 1:
                possible_byte = get_possible_byte(jis_fixed[:i], jis_fixed[i+1])
            else:
                possible_byte = get_possible_byte(jis_fixed[:i])
            jis_fixed = jis_fixed[:i] + possible_byte[0] + jis_fixed[i:]
            fo_buf += possible_byte[0]
            i += 1

    elif len(fo_buf[-1]) == 3:
        try:
            c = fo_buf.decode('utf-8')
            assert c in TUDOU or c in BYTEMARK
        except:
            # something wrong, may be lack of byte